In [39]:
"""
Evaluate linear combination of Legendre polynomials L_k(x) given in point x from 1d interval.
In short, f(x) = sum_k c_k L_k(x) is computed.
"""
function legval{T<:Real}(x::T, c::Vector{T})
  if length(c) == 1
     c0 = c[1]
     c1 = 0.
  elseif length(c) == 2
      c0 = c[1]
      c1 = c[2]
  else
    nd = length(c)
    c0 = c[end-1]
    c1 = c[end]

    for i in range(2, nd-2)
      tmp = c0
      nd = nd - 1
      c0 = c[end-i] - (c1*(nd - 1))/nd
      c1 = tmp + (c1*x*(2*nd - 1))/nd
    end  
  end
  return c0 + c1*x
end

"""
Compute m-th derivative of Legendre series as another Legendre series.
So legval(x, legder(c, 1)) is the value at x of sum_k c_k L'_k.
"""
function legder(c::Vector{Float64}, m::Int64)
  assert(m >= 0)
  # 0 order derivative
  if m == 0
    return c
  # len(c) is 0, 1, 2, 3 degree. Check for derivative of order larger than max
  # degree
  elseif m >= length(c)
    return [0.]
  # Compute 
  else
    n = length(c)
    for i in range(1, m)
      n = n - 1
      der = zeros(n)
      c = copy(c)
     
      for j in n:-1:3
        der[j] = (2*j - 1)*c[j+1]
        c[j - 1] += c[j+1]
      end
      if n > 1
        der[2] = 3*c[3]
      end
      der[1] = c[2]
      c = der
    end
  end
  return c
end

"""
Vectorized version of legval. X is a vector of x coordinates.
"""
function legval{T<:Real}(x::Vector{T}, c::Vector{T})
    y = similar(x)
    for (i, xi) in enumerate(x)
        y[i] = legval(xi, c)
    end
    y
end

legval (generic function with 4 methods)

In [41]:
# Test against sympy
c = [1., 2., 3., -1.]
points = [0.2, 0.1, 0.5, 0.6]

values = legval(points, c)
for (x, v) in zip(points, values)
    println("x=$(x) ", 
            "f=$(v) ",
            "df=$(legval(x, legder(c, 1))) ",
            "d^2f=$(legval(x, legder(c, 2)))")
end

x=0.2 f=0.3600000000000001 df=5.0 d^2f=6.0
x=0.1 f=-0.10750000000000004 df=4.325 d^2f=7.5
x=0.5 f=2.0625 df=6.125 d^2f=1.5
x=0.6 f=2.68 df=6.199999999999999 d^2f=0.0


In [44]:
"""
Evaluate linear combination of Legendre polynomials L_i(x)*L_j(y) given in point x from 2d.
The polynomials are obtained as tensor product of 1d polynomial in x and y, i.e.
P(x, y) = c[0, 0]*L_0(x)*L_0(x) + c[0, 1]*L_0(x)*L_1(y) ...
"""
function legval{T<:Real}(x::T, y::T, c::Matrix{T})
    # First eval at x coord polynomials of x specified by columns in c
    ncols = size(c, 2)
    c_x = zeros(T, ncols)
    for col in 1:ncols
        c_x[col] = legval(x, c[:, col])
    end
    # Now eval at y with new coefs
    legval(y, c_x)
end

"""
Compute derivatives of 2d(tensor product) Legendre series. The derivative is specified
by a multidex m = [mx, my] and requests mx derivative w.r.t to x and my derivative w.r.t
y.
"""
function legder{T<:Real}(c::Matrix{T}, m::Vector{Int64})
    assert(all(v -> v >= 0, m))   # No antiderivativec
    assert(length(m) == 2)        # Multiindex of length 2
    # Zero-th order derivative
    if m == [0, 0]
        return c
    # Partial derivative w.r.t x
    elseif m == [1, 0]
        # Handle function independent of x
        nrows, ncols = size(c, 1), size(c, 2)
        if nrows == 1
            return zeros(c)
        end
        
        # Take derivative col by col
        diff = zeros(nrows-1, ncols)
        for col in 1:ncols
            diff[:, col] = legder(c[:, col])
        end
        return diff
    # Partial derivative w.r.t y 
    elseif m == [0, 1]
        # Clever use of recursion :)
        return legder(c', [1, 0])'
    # Higher order stuff
    else
        mx, my = first(m), last(m)
        coef = copy(c)
        # We take the sweps in x and y. This should be more stable then loweing 
        # x and then y.
        while mx > 0 || my > 0
            # \partial x
            if mx > 0
                coef = legder(coef, [1, 0])
                mx -= 1
            end
            # \partial y
            if my > 0
                coef = legder(coef, [0, 1])
                my -= 1
            end
        end
        return coef
    end
end

"""
Vectorized version of legval. X is a (number of points, 2) matrix of coordinates.
"""
function legval{T<:Real}(x::Matrix{T}, c::Matrix{T})
    nrows = size(x, 1)
    y = zeros(T, nrows)
    for i in 1:nrows
        y[i] = legval(x[i, 1], x[i, 2], c)
    end
    y
end

legval (generic function with 4 methods)

In [43]:
# Test against sympy
c = [1. 2. -1; 3 4 0.2]
points = [0.3 0.4; -0.2 0.6; 0.8 -1.]

f = legval(points, c)
dx = legval(points, legder(c, [1, 0]))
dy = legval(points, legder(c, [0, 1]))
dxdx = legval(points, legder(c, [2, 0]))
dxdy = legval(points, legder(c, [1, 1]))
dydy = legval(points, legder(c, [0, 2]))

println(f)
println(dx)
println(dy)
println(dxdx)
println(dxdy)
println(dydy)

[3.4244000000000003,1.0784,-2.6399999999999997]
[4.548,5.4079999999999995,-0.8000000000000003]
[2.072,-0.6719999999999999,7.720000000000001]
[0.0,0.0,0.0]
[4.24,4.36,3.4]
[-2.82,-3.12,-2.52]


In [85]:
typealias MatVec Union{Vector, Matrix}

# Legendre polynomial is given by expansion coefs w.r.t basis of L_k
type LegendrePolynomial
    coefs::MatVec
end

# Evaluate legendre polynomial
function Base.call(f::LegendrePolynomial, x::MatVec)
    10
end

call (generic function with 1090 methods)

In [83]:
methods(Array)

7-element Array{Any,1}:
 call{T,N}(::Type{Array{T,N}}, ::Type{T}, d::NTuple{N,Int64}) at essentials.jl:198              
 call{T}(::Type{Array{T,N}}, ::Type{T}, m::Integer) at essentials.jl:200                        
 call{T}(::Type{Array{T,N}}, ::Type{T}, m::Integer, n::Integer) at essentials.jl:201            
 call{T}(::Type{Array{T,N}}, ::Type{T}, m::Integer, n::Integer, o::Integer) at essentials.jl:202
 call{T}(::Type{Array{T,N}}, ::Type{T}, d::Integer...) at essentials.jl:199                     
 call{T}(::Type{T}, arg) at essentials.jl:56                                                    
 call{T}(::Type{T}, args...) at essentials.jl:57                                                

In [71]:
vec = [1, 2]
mat = [1 2; 3 4]

2x2 Array{Int64,2}:
 1  2
 3  4

In [73]:
fooo(mat)

4

In [88]:
type Miro
end

Base.call(::Miro, x) = 1+x

call (generic function with 1093 methods)

In [89]:
map(Miro(), [1, 2, 3])

3-element Array{Int64,1}:
 2
 3
 4

In [64]:
Function

Function

In [65]:
type XX <:
end

LoadError: LoadError: invalid subtyping in definition of XX
while loading In[65], in expression starting on line 1